In [68]:
import sys
import mysql.connector

def connect(user):
    conn = mysql.connector.connect(host='localhost', database='microshop', user=user)
    conn.autocommit = True
    return conn

rootconn = connect('user')


def sqlQuery(sqlString, conn=rootconn):
    try:
        cursor = rootconn.cursor()
        cursor.execute(sqlString)
        res = cursor.fetchall()
        return res
    except Exception as ex:
        print(str(ex), file=sys.stderr)
    finally:    
        cursor.close()

def sqlDo(sqlString, conn=rootconn):
    try:
        cursor = conn.cursor()
        cursor.execute(sqlString)
        res = cursor.fetchwarnings()
        return res
    except Exception as ex:
        print(str(ex), file=sys.stderr)
    finally:    
        cursor.close()

In [69]:
sqlQuery("show tables")

[('customer',), ('order',), ('orderline',), ('product',)]

In [85]:
sqlQuery("INSERT INTO `microshop`.`customer` (`name`) VALUES ('Joe');");


No result set to fetch from.


In [70]:
def myQuery(query):
    q = f"select * from {query}"
    print(q)
    return sqlQuery(q)



In [86]:
#return all customers

customers = myQuery("customer")
for customer in customers:
    print(customer)

select * from customer
(1, 'Bob')
(2, 'Dave')
(3, 'Farkas')
(4, 'Arkadius')
(5, 'Vlad')
(6, 'Joe')
(7, 'Doe')


In [79]:
#Updating myQuery

def myQuery1(query):
    res = query.split('.')
    if(len(res) < 2):
        q = f"select * from {query.lower()}"
        return sqlQuery(q) 
    else:
        q = f"select {res[1].lower()} from {res[0].lower()}"
        print(q)
        return sqlQuery(q)




In [83]:
#return all customer names

names = myQuery1("Customer.name")
for name in names:
    print(name)

select name from customer
('Bob',)
('Dave',)
('Farkas',)
('Arkadius',)
('Vlad',)


In [84]:
# Funky idea
#(Customer|name='Joe') , can probably be extended to the rest

def myQuery2(query):
    t = query.replace('(','').replace('|', "").replace(')',"").replace("name=", " ").replace('.',' ')
    tlist = t.split()
    return sqlQuery(f"select * from {tlist[0].lower()} where name={tlist[1]}")


    
myQuery2("(Customer|name='Joe').Orders.OrderLine.Product")


[(5, 'Vlad')]